In [37]:
import decimal
import datetime
from enum import Enum
from typing import Dict, Any, List, Optional, Union
from abc import ABC, abstractmethod

class KeyType(Enum):
    """Enumeration representing key types for SQL column definitions."""
    PRIMARY_KEY = "PRIMARY KEY"
    UNIQUE = "UNIQUE"
    FOREIGN_KEY = "FOREIGN KEY"
    CHECK = "CHECK"
    
    def __str__(self):
        return self.value
    
class ReferenceOption(Enum):
    """Enumeration representing reference options for FOREIGN KEY constraints."""
    CASCADE = "CASCADE"
    RESTRICT = "RESTRICT"
    NO_ACTION = "NO ACTION"
    SET_NULL = "SET NULL"
    SET_DEFAULT = "SET DEFAULT"
    AUTO_INCREMENT = "AUTO_INCREMENT"
    
    def __str__(self):
        return self.value

class JoinType(Enum):
    """Enumeration representing join types for SQL JOIN queries."""
    INNER = "INNER"
    LEFT = "LEFT"
    RIGHT = "RIGHT"
    FULL = "FULL"
    
    def __str__(self):
        return self.value

class SQL_Type(ABC):
    @abstractmethod
    def __str__(self):
        pass

class SQL_Varchar(SQL_Type):
    def __init__(self, length: int):
        if not isinstance(length, int):
            raise SQLValueError("Invalid length for VARCHAR type. Length must be an integer.")
        if not (1 <= length <= 255):
            raise SQLValueError("Invalid length for VARCHAR type. Length must be between 1 and 255.")
        self._length = length
    
    def __str__(self):
        return f"VARCHAR({self._length})"

class SQL_Decimal(SQL_Type):
    def __init__(self, precision: int=10, scale: int=2):
        if not (1 <= precision <= 38 and 0 <= scale <= precision):
            raise SQLValueError("Invalid precision and scale for DECIMAL type. Precision must be between 1 and 38, and scale must be between 0 and precision.")
        self._precision = precision
        self._scale = scale
    
    def __str__(self):
        return f'DECIMAL({self._precision, self._scale})'

def to_sql_type(py_type: type) -> str:
    """
    Converts a Python data type to the corresponding SQL data type.

    Args:
        py_type (type): The Python data type.

    Returns:
        str: The SQL data type.
    """

    type_map: Dict[type, str] = {
        int: "INT",
        float: "FLOAT",
        str: "VARCHAR(255)",
        bytes: "BLOB",
        bool: "BOOL",
        datetime.date: "DATE",
        datetime.datetime: "DATETIME",
        decimal.Decimal: "DECIMAL(10, 4)"
    }
    
    return type_map.get(py_type, "VARCHAR(255)")  # Default to VARCHAR(255)

def column(col_name: str, py_type: type, key_type: Optional[KeyType] = None, reference: Optional[List[str]] = None, on_delete: Optional[str] = None) -> str:
    """
    Generates a SQL column definition string.

    Args:
        col_name (str): The column name.
        py_type (type): The Python data type of the column.
        key_type (Optional[KeyType]): The key type for the column (e.g., PRIMARY KEY, UNIQUE, etc.).
        reference (Optional[List[str]]): The reference table and column for FOREIGN KEY constraints.
        on_delete (Optional[str]): The reference option for FOREIGN KEY constraints (e.g., CASCADE, SET NULL, etc.).

    Returns:
        str: The SQL column definition string.
    """
    if isinstance(py_type, SQL_Type):
        col_type: str = f'{py_type}'
    else:
        col_type: str = to_sql_type(py_type)
    col_def: str = f"{col_name} {col_type}"

    if key_type:
        if key_type in [KeyType.PRIMARY_KEY, KeyType.UNIQUE]:
            col_def += f" {key_type}"
        elif key_type == KeyType.FOREIGN_KEY and reference:
            ref_table, ref_column = reference
            on_delete_str = f" ON DELETE {on_delete}" if on_delete else ""
            col_def += f" {key_type} ({col_name}) REFERENCES {ref_table} ({ref_column}){on_delete_str}"

    return col_def

class SQLValueError(Exception):
    """
    Custom exception raised when duplicate column names are found in the table definition.
    """
    def __init__(self, message: str) -> None:
        super().__init__(message)

class DuplicateColumnError(Exception):
    """
    Custom exception raised when duplicate column names are found in the table definition.
    """
    def __init__(self, message: str) -> None:
        super().__init__(message)

class InvalidColumnError(Exception):
    """
    Custom exception raised when invalid column names are used in queries.
    """
    def __init__(self, message: str) -> None:
        super().__init__(message)

class Table:
    """
    Represents a table in the database.

    Attributes:
        table_name (str): The name of the table.
        _columns (Dict[str, List[Union[str, type, Optional[KeyType], Optional[str], Optional[str]]]]):
            The columns of the table with their definitions.

    Args:
        table_name (str): The name of the table.
        columns (Dict[str, List[Union[str, type, Optional[KeyType], Optional[str], Optional[str]]]]):
            The columns of the table with their definitions.
    """
    def __init__(self, table_name: str, columns: Dict[str, List[Union[str, type, Optional[KeyType], Optional[str], Optional[str]]]]) -> None:
        """
        Initializes a new table instance.

        Args:
            table_name (str): The name of the table.
            columns (Dict[str, List[Union[str, type, Optional[KeyType], Optional[str], Optional[str]]]]):
                The columns of the table with their definitions.

        Raises:
            DuplicateColumnError: If duplicate column names are found in the table definition.
        """
        # Check for duplicate column names
        column_names = list(columns.keys())
        no_duplicate_columns = set(column_names)
        if len(no_duplicate_columns) < len(column_names):
            duplicate_columns = [name for name in column_names if column_names.count(name) > 1]
            duplicate_columns_str = ', '.join(duplicate_columns)
            raise DuplicateColumnError(f"Duplicate column name(s) found in table definition: {duplicate_columns_str}")

        # If no duplicate columns, then assign attributes
        self.table_name = table_name
        self._columns = columns

    def create(self) -> str:
        """
        Generates a SQL CREATE TABLE query.

        Returns:
            str: The SQL CREATE TABLE query.
        """
        col_defs_str = ', '.join([column(col_name, *col_def) for col_name, col_def in self._columns.items()])
        return f"CREATE TABLE IF NOT EXISTS {self.table_name} ({col_defs_str});"

    def insert(self, data: Dict[str, Any]) -> str:
        """
        Generates a SQL INSERT query.

        Args:
            data (Dict[str, Any]): The data to be inserted into the table.

        Returns:
            str: The SQL INSERT query.

        Raises:
            InvalidColumnError: If invalid column names are used in the INSERT query.
        """
        invalid_columns = set(data.keys()) - set(self.columns())
        if invalid_columns:
            invalid_columns_str = ', '.join(invalid_columns)
            raise InvalidColumnError(f"Invalid column{'s' if len(invalid_columns) > 1 else ''} in INSERT query: {invalid_columns_str}")

        keys = ', '.join(data.keys())
        values = ', '.join([f"'{val}'" for val in data.values()])
        return f"INSERT INTO {self.table_name} ({keys}) VALUES ({values});"

    def select(self, columns: Optional[List[str]] = None, conditions: Optional[Dict[str, Any]] = None) -> str:
        """
        Generates a SQL SELECT query.

        Args:
            columns (Optional[List[str]]): The columns to be selected. If None, selects all columns.
            conditions (Optional[Dict[str, Any]]): The conditions to filter the query. If None, no conditions are applied.

        Returns:
            str: The SQL SELECT query.

        Raises:
            InvalidColumnError: If invalid column names are used in the SELECT query.
        """
        col_names = self.columns()
        if columns:
            invalid_columns = set(columns) - set(col_names)
            if invalid_columns:
                invalid_columns_str = ', '.join(invalid_columns)
                raise InvalidColumnError(f"Invalid column{'s' if len(invalid_columns) > 1 else ''} in SELECT query: {invalid_columns_str}")

        if not conditions:
            return f"SELECT {', '.join(columns) if columns else '*'} FROM {self.table_name};"
        else:
            invalid_columns = set(conditions.keys()) - set(col_names)
            if invalid_columns:
                invalid_columns_str = ', '.join(invalid_columns)
                raise InvalidColumnError(f"Invalid column{'s' if len(invalid_columns) > 1 else ''} in SELECT query conditions: {invalid_columns_str}")

            conditions_str = ' AND '.join([f"{key}='{value}'" for key, value in conditions.items()])
            return f"SELECT {', '.join(columns) if columns else '*'} FROM {self.table_name} WHERE {conditions_str};"

    def update(self, data: Dict[str, Any], conditions: Optional[Dict[str, Any]] = None) -> str:
        """
        Generates a SQL UPDATE query.

        Args:
            data (Dict[str, Any]): The data to be updated.
            conditions (Optional[Dict[str, Any]]): The conditions to filter the update.

        Returns:
            str: The SQL UPDATE query.

        Raises:
            InvalidColumnError: If invalid column names are used in the UPDATE query.
        """
        col_names = self.columns()
        invalid_columns = set(data.keys()) - set(col_names)

        if invalid_columns:
            invalid_columns_str = ', '.join(invalid_columns)
            raise InvalidColumnError(f"Invalid column{'s' if len(invalid_columns) > 1 else ''} in UPDATE query: {invalid_columns_str}")

        update_str = ', '.join([f"{key}='{value}'" for key, value in data.items()])
        conditions_str = ' AND '.join([f"{key}='{value}'" for key, value in conditions.items()]) if conditions else ""
        return f"UPDATE {self.table_name} SET {update_str} WHERE {conditions_str};"

    def delete(self, conditions: Optional[Dict[str, Any]] = None) -> str:
        """
        Generates a SQL DELETE query.

        Args:
            conditions (Optional[Dict[str, Any]]): The conditions to filter the delete.

        Returns:
            str: The SQL DELETE query.

        Raises:
            InvalidColumnError: If invalid column names are used in the DELETE query.
        """
        col_names = self.columns()
        if conditions:
            invalid_columns = set(conditions.keys()) - set(col_names)
            if invalid_columns:
                invalid_columns_str = ', '.join(invalid_columns)
                raise InvalidColumnError(f"Invalid column{'s' if len(invalid_columns) > 1 else ''} in DELETE query conditions: {invalid_columns_str}")

        conditions_str = ' AND '.join([f"{key}='{value}'" for key, value in conditions.items()]) if conditions else ""
        return f"DELETE FROM {self.table_name} WHERE {conditions_str};"

    def join(self, other_table: 'Table', on_column: str, join_type: JoinType) -> str:
        """
        Generates a SQL JOIN query.

        Args:
            other_table (Table): The other table to join with.
            on_column (str): The column to join on.
            join_type (JoinType): The type of JOIN to perform.

        Returns:
            str: The SQL JOIN query.

        Raises:
            InvalidColumnError: If invalid column names are used in the JOIN query.
        """
        col_names = self.columns()
        if on_column not in col_names or on_column not in other_table.columns():
            raise InvalidColumnError(f"Invalid column in JOIN query: {on_column}")

        other_table_name = other_table.table_name
        return f"{join_type} JOIN {other_table_name} ON {self.table_name}.{on_column} = {other_table_name}.{on_column};"

    def columns(self) -> List[str]:
        """
        Get the names of all columns in the table.

        Returns:
            List[str]: The list of column names.
        """
        return list(self._columns.keys())


In [40]:
#from pydbhelper import Table, KeyType, ReferenceOption, JoinType

def main():
    # Define table schemas
    users_table = Table("users", {
        "id": [int, KeyType.PRIMARY_KEY],
        "name": [SQL_Varchar(50)],
        "email": [SQL_Varchar(100), KeyType.UNIQUE],
        "account_id": [int, KeyType.FOREIGN_KEY, ["accounts", "id"], ReferenceOption.CASCADE]
    })

    accounts_table = Table("accounts", {
        "id": [int, KeyType.PRIMARY_KEY],
        "account_name": [SQL_Varchar(50)],
        "balance": [SQL_Decimal(10,2)],
    })

    characters_table = Table("characters", {
        "id": [int, KeyType.PRIMARY_KEY],
        "name": [SQL_Varchar(50)],
        "class": [SQL_Varchar(20)],
        "user_id": [int, KeyType.FOREIGN_KEY, ["users", "id"], ReferenceOption.CASCADE]
    })

    character_stats_table = Table("character_stats", {
        "id": [int, KeyType.PRIMARY_KEY],
        "strength": [int],
        "dexterity": [int],
        "constitution": [int],
        "character_id": [int, KeyType.FOREIGN_KEY, ["characters", "id"], ReferenceOption.CASCADE]
    })

    
    # Generate SQL queries for table creation
    users_create_query = users_table.create()
    accounts_create_query = accounts_table.create()
    characters_create_query = characters_table.create()
    character_stats_create_query = character_stats_table.create()

    # Print SQL queries
    print("SQL Queries for Table Creation:")
    print(users_create_query)
    print(accounts_create_query)
    print(characters_create_query)
    print(character_stats_create_query)

    # Example data for table insertion
    user_data = {
        "id": 1,
        "name": "John Doe",
        "email": "john.doe@example.com",
        "account_id": 1001
    }

    account_data = {
        "id": 1001,
        "account_name": "Savings",
        "balance": 5000.00
    }

    character_data = {
        "id": 1,
        "name": "Eldric",
        "class": "Wizard",
        "user_id": 1
    }

    character_stats_data = {
        "id": 1,
        "strength": 12,
        "dexterity": 14,
        "constitution": 16,
        "character_id": 1
    }

    # Generate SQL queries for data insertion
    user_insert_query = users_table.insert(user_data)
    account_insert_query = accounts_table.insert(account_data)
    character_insert_query = characters_table.insert(character_data)
    character_stats_insert_query = character_stats_table.insert(character_stats_data)

    # Print SQL queries
    print("\nSQL Queries for Data Insertion:")
    print(user_insert_query)
    print(account_insert_query)
    print(character_insert_query)
    print(character_stats_insert_query)

    # Generate SQL query for data retrieval
    select_query = characters_table.select(columns=["name", "class"], conditions={"id": 1})

    # Print SQL query
    print("\nSQL Query for Data Retrieval:")
    print(select_query)

    # Generate SQL query for data update
    update_data = {"class": "Sorcerer"}
    update_query = characters_table.update(data=update_data, conditions={"id": 1})

    # Print SQL query
    print("\nSQL Query for Data Update:")
    print(update_query)

    # Generate SQL query for data deletion
    delete_query = character_stats_table.delete(conditions={"id": 1})

    # Print SQL query
    print("\nSQL Query for Data Deletion:")
    print(delete_query)

    # Generate SQL query for joining tables
    join_query = users_table.join(other_table=characters_table, on_column="id", join_type=JoinType.INNER)

    # Print SQL query
    print("\nSQL Query for Joining Tables:")
    print(join_query)

if __name__ == "__main__":
    main()

SQL Queries for Table Creation:
CREATE TABLE IF NOT EXISTS users (id INT PRIMARY KEY, name VARCHAR(50), email VARCHAR(100) UNIQUE, account_id INT FOREIGN KEY (account_id) REFERENCES accounts (id) ON DELETE CASCADE);
CREATE TABLE IF NOT EXISTS accounts (id INT PRIMARY KEY, account_name VARCHAR(50), balance DECIMAL((10, 2)));
CREATE TABLE IF NOT EXISTS characters (id INT PRIMARY KEY, name VARCHAR(50), class VARCHAR(20), user_id INT FOREIGN KEY (user_id) REFERENCES users (id) ON DELETE CASCADE);
CREATE TABLE IF NOT EXISTS character_stats (id INT PRIMARY KEY, strength INT, dexterity INT, constitution INT, character_id INT FOREIGN KEY (character_id) REFERENCES characters (id) ON DELETE CASCADE);

SQL Queries for Data Insertion:
INSERT INTO users (id, name, email, account_id) VALUES ('1', 'John Doe', 'john.doe@example.com', '1001');
INSERT INTO accounts (id, account_name, balance) VALUES ('1001', 'Savings', '5000.0');
INSERT INTO characters (id, name, class, user_id) VALUES ('1', 'Eldric', '

In [94]:
import pandas as pd
from peewee import *

# Define a SQLite database
db = SqliteDatabase('example.db')

class Accounts(Model):
    id = AutoField(primary_key=True)
    account_name = CharField(max_length=50)
    balance = DecimalField(max_digits=10, decimal_places=2)

    class Meta:
        database = db

class Users(Model):
    id = AutoField(primary_key=True)
    name = CharField(max_length=50)
    email = CharField(max_length=100, unique=True)
    account_id = ForeignKeyField(Accounts, backref='users', on_delete='CASCADE')

    class Meta:
        database = db

class Characters(Model):
    id = AutoField(primary_key=True)
    name = CharField(max_length=50)
    class_name = CharField(max_length=20)
    user_id = ForeignKeyField(Users, backref='characters', on_delete='CASCADE')

    class Meta:
        database = db

class CharacterStats(Model):
    id = AutoField(primary_key=True)
    strength = IntegerField()
    dexterity = IntegerField()
    constitution = IntegerField()
    character_id = ForeignKeyField(Characters, backref='character_stats', on_delete='CASCADE')

    class Meta:
        database = db

def main():
    # Connect to the database
    db.connect()

    # Query data from tables using Peewee's get and select methods
    characters_data = Characters.select().where(Characters.name == 'Eldric')

    # Convert the characters data to a pandas DataFrame
    characters_df = pd.DataFrame(list(characters_data.dicts()))

    # Close the database connection
    db.close()

    # Display the character's information using pandas
    print(characters_df)

if __name__ == "__main__":
    main()


    id    name class_name  user_id
0    1  Eldric   Sorcerer        1
1    2  Eldric     Wizard        1
2    3  Eldric     Wizard        1
3    4  Eldric     Wizard        1
4    5  Eldric     Wizard        1
5    6  Eldric     Wizard        1
6    7  Eldric     Wizard        1
7    8  Eldric     Wizard        1
8    9  Eldric     Wizard        1
9   10  Eldric     Wizard        1
10  11  Eldric     Wizard        1
11  12  Eldric     Wizard        1
12  13  Eldric     Wizard        1
13  14  Eldric     Wizard        1
14  15  Eldric     Wizard        1
15  16  Eldric     Wizard        1
16  17  Eldric     Wizard        1
17  18  Eldric     Wizard        1
18  19  Eldric     Wizard        1
19  20  Eldric     Wizard        1
20  21  Eldric     Wizard        1
21  22  Eldric     Wizard        1
22  23  Eldric     Wizard        1
23  24  Eldric     Wizard        1
24  25  Eldric     Wizard        1
25  26  Eldric     Wizard        1
26  27  Eldric     Wizard        1
27  28  Eldric     W